In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline

В данном примере будет регрессионная модель, на примере предсказания стоимости машины

In [2]:
data=pd.read_csv('/content/CarPrice_Assignment.csv')

Выведем  информацию о датасете

In [ ]:
data.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1

Удаляем столбецы car_ID и carName, т.к они нам не нужны

In [3]:
data.drop(['car_ID','CarName'],axis = 1,inplace = True)

In [ ]:
data.size

5330

In [ ]:
data.isnull().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

In [ ]:
px.imshow(data.corr(),title="Корреляционная матрица")

<ipython-input-10-e94a664d1193>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  px.imshow(data.corr(),title="Корреляционная матрица")


Переводим все строки в категории

In [4]:
from sklearn.preprocessing import LabelEncoder

labelsToEncode=['fueltype','aspiration','doornumber','carbody','drivewheel','enginelocation','enginetype','cylindernumber','fuelsystem']
for label in labelsToEncode:
  le = LabelEncoder()
  data[label] = le.fit_transform(data[label])

In [5]:
data

,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,3,1,0,1,0,2,0,88.6,168.8,64.1,...,130,5,3.47,2.68,9.0,111,5000,21,27,13495.0
1,3,1,0,1,0,2,0,88.6,168.8,64.1,...,130,5,3.47,2.68,9.0,111,5000,21,27,16500.0
2,1,1,0,1,2,2,0,94.5,171.2,65.5,...,152,5,2.68,3.47,9.0,154,5000,19,26,16500.0
3,2,1,0,0,3,1,0,99.8,176.6,66.2,...,109,5,3.19,3.40,10.0,102,5500,24,30,13950.0
4,2,1,0,0,3,0,0,99.4,176.6,66.4,...,136,5,3.19,3.40,8.0,115,5500,18,22,17450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,1,0,0,3,2,0,109.1,188.8,68.9,...,141,5,3.78,3.15,9.5,114,5400,23,28,16845.0
201,-1,1,1,0,3,2,0,109.1,188.8,68.8,...,141,5,3.78,3.15,8.7,160,5300,19,25,19045.0
202,-1,1,0,0,3,2,0,109.1,188.8,68.9,...,173,5,3.58,2.87,8.8,134,5500,18,23,21485.0
203,-1,0,1,0,3,2,0,109.1,188.8,68.9,...,145,3,3.01,3.40,23.0,106,4800,26,27,22470.0


Одним из этапов предворительной обработки данных является стандартизация значений, она используется, когда значения в большом диапазоне, для упращение тренировки модели диапазон значений уменьшается

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,confusion_matrix
scaler = StandardScaler()
scaler.fit(data.drop(['price'],axis = 1))
scaled_features = scaler.transform(data.drop('price',axis = 1))

In [14]:
scaled_features[0]

array([ 1.74347043,  0.32879797, -0.46929532,  1.13038833, -3.05097525,
        1.21333041, -0.12186667, -1.6907718 , -0.42652147, -0.84478235,
       -2.0204173 , -0.01456628, -2.86510549, -0.14747536,  0.07444893,
        0.8695675 ,  0.51907138, -1.83937734, -0.28834891,  0.17448278,
       -0.26296022, -0.64655303, -0.54605874])

Перед тренировкой модели датасет разбивается на 2 части, на тренировочный и  на тестовый датасеты. Это делается для того, что бы после окончания тренировки модели проверить ее точность на данных, которая модель не видела

In [15]:
from sklearn.model_selection import train_test_split
X = scaled_features
Y = data['price']
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.3,random_state=44)

Ниже предоставленны несколько моделей и их метрики, для определения точности модели использовались следующие метрики


1.   Среднеквадратичная ошибка
2.   Среднеквадратичная ошибка в процентах


Линейная регрессия

In [36]:
from sklearn.metrics import mean_squared_error#– Среднее арифметическое  квадратов разностей между предсказанными и реальными значениями
from sklearn.metrics import mean_absolute_percentage_error#относительная ошибка, где разность между наблюдаемым и фактическим значениями делится на наблюдаемое значение и выражается в процентах:

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV

In [17]:

reg=LinearRegression()
reg.fit(X_train,Y_train)

LinearRegression()

In [18]:
pred = reg.predict(X_test)

In [19]:
mean_squared_error(Y_test, pred)

12039658.561036099

In [28]:
mean_absolute_percentage_error(Y_test, pred)

0.18409590499084277

Случайный лес

In [ ]:
regr = RandomForestRegressor(max_depth=4, random_state=0)
regr.fit(X_train, Y_train)

In [32]:
pred2 = regr.predict(X_test)

In [33]:
mean_squared_error(Y_test, pred2)

5293070.890525037

In [34]:
mean_absolute_percentage_error(Y_test, pred2)

0.12910243193564508

In [38]:
lassoReg = LassoCV(cv=5, random_state=0).fit(X_train, Y_train)

In [39]:
pred3 = lassoReg.predict(X_test)

In [40]:
mean_squared_error(Y_test, pred3)

8870986.405887594

In [41]:
mean_absolute_percentage_error(Y_test, pred3)

0.15898824677762305